In [1000]:
#LOAD PACKAGES

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from feature_engine import missing_data_imputers as mdi
from feature_engine import discretisers as dsc
from feature_engine import categorical_encoders as ce
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.datasets import load_boston
from feature_engine.outlier_removers import Winsorizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import sklearn.metrics as metrics
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [1001]:
# load dataset
data = pd.read_csv('houseprices.csv')

print(data.shape)

data.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Basic Info:
Let's find out what types of variables there are in this dataset, statistical details, nulls, etc

In [1002]:
# info to understand nulls & data types.
# clearly need to drop few columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [1003]:
# drop columns with alot of nulls
data = data.dropna(thresh=data.shape[0]*0.8,how='all',axis=1)

In [1004]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

# Feature Engineering

In [1005]:
# let's separate into training and testing set


X_train, X_test, y_train, y_test = train_test_split(data.drop(['Id', 'SalePrice'], axis=1),
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)

X_train.shape, X_test.shape

((1022, 74), (438, 74))

In [1006]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 64 to 684
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1022 non-null   int64  
 1   MSZoning       1022 non-null   object 
 2   LotFrontage    833 non-null    float64
 3   LotArea        1022 non-null   int64  
 4   Street         1022 non-null   object 
 5   LotShape       1022 non-null   object 
 6   LandContour    1022 non-null   object 
 7   Utilities      1022 non-null   object 
 8   LotConfig      1022 non-null   object 
 9   LandSlope      1022 non-null   object 
 10  Neighborhood   1022 non-null   object 
 11  Condition1     1022 non-null   object 
 12  Condition2     1022 non-null   object 
 13  BldgType       1022 non-null   object 
 14  HouseStyle     1022 non-null   object 
 15  OverallQual    1022 non-null   int64  
 16  OverallCond    1022 non-null   int64  
 17  YearBuilt      1022 non-null   int64  
 18  YearRemo

In [1007]:
X_ob = X_train.select_dtypes(include=['object'])


In [1008]:
X_ob.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 64 to 684
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1022 non-null   object
 1   Street         1022 non-null   object
 2   LotShape       1022 non-null   object
 3   LandContour    1022 non-null   object
 4   Utilities      1022 non-null   object
 5   LotConfig      1022 non-null   object
 6   LandSlope      1022 non-null   object
 7   Neighborhood   1022 non-null   object
 8   Condition1     1022 non-null   object
 9   Condition2     1022 non-null   object
 10  BldgType       1022 non-null   object
 11  HouseStyle     1022 non-null   object
 12  RoofStyle      1022 non-null   object
 13  RoofMatl       1022 non-null   object
 14  Exterior1st    1022 non-null   object
 15  Exterior2nd    1022 non-null   object
 16  MasVnrType     1017 non-null   object
 17  ExterQual      1022 non-null   object
 18  ExterCond      1022 non-null

In [1009]:
X_ob.fillna('Missing', inplace=True)

C:\Users\dhruv\Anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [1010]:
X_ob.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 64 to 684
Data columns (total 38 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1022 non-null   object
 1   Street         1022 non-null   object
 2   LotShape       1022 non-null   object
 3   LandContour    1022 non-null   object
 4   Utilities      1022 non-null   object
 5   LotConfig      1022 non-null   object
 6   LandSlope      1022 non-null   object
 7   Neighborhood   1022 non-null   object
 8   Condition1     1022 non-null   object
 9   Condition2     1022 non-null   object
 10  BldgType       1022 non-null   object
 11  HouseStyle     1022 non-null   object
 12  RoofStyle      1022 non-null   object
 13  RoofMatl       1022 non-null   object
 14  Exterior1st    1022 non-null   object
 15  Exterior2nd    1022 non-null   object
 16  MasVnrType     1022 non-null   object
 17  ExterQual      1022 non-null   object
 18  ExterCond      1022 non-null

In [1011]:
X_num = X_train.select_dtypes(exclude=['object'])
X_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 64 to 684
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1022 non-null   int64  
 1   LotFrontage    833 non-null    float64
 2   LotArea        1022 non-null   int64  
 3   OverallQual    1022 non-null   int64  
 4   OverallCond    1022 non-null   int64  
 5   YearBuilt      1022 non-null   int64  
 6   YearRemodAdd   1022 non-null   int64  
 7   MasVnrArea     1017 non-null   float64
 8   BsmtFinSF1     1022 non-null   int64  
 9   BsmtFinSF2     1022 non-null   int64  
 10  BsmtUnfSF      1022 non-null   int64  
 11  TotalBsmtSF    1022 non-null   int64  
 12  1stFlrSF       1022 non-null   int64  
 13  2ndFlrSF       1022 non-null   int64  
 14  LowQualFinSF   1022 non-null   int64  
 15  GrLivArea      1022 non-null   int64  
 16  BsmtFullBath   1022 non-null   int64  
 17  BsmtHalfBath   1022 non-null   int64  
 18  FullBath

In [1012]:
X_num = X_num.fillna(X_num.mean())
X_num.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
64,60,69.668667,9375,7,5,1997,1998,573.0,739,0,...,645,576,36,0,0,0,0,0,2,2009
682,120,69.668667,2887,6,5,1996,1997,0.0,1003,0,...,431,307,0,0,0,0,0,0,11,2008
960,20,50.000000,7207,5,7,1958,2008,0.0,696,0,...,0,117,0,0,0,0,0,0,2,2010
1384,50,60.000000,9060,6,5,1939,1950,0.0,204,0,...,280,0,0,0,0,0,0,0,10,2009
1100,30,60.000000,8400,2,5,1920,1950,0.0,290,0,...,246,0,0,0,0,0,0,0,1,2009


In [1013]:
X_train = pd.concat([X_num,X_ob],axis=1,sort=False)


In [1014]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 64 to 684
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1022 non-null   int64  
 1   LotFrontage    1022 non-null   float64
 2   LotArea        1022 non-null   int64  
 3   OverallQual    1022 non-null   int64  
 4   OverallCond    1022 non-null   int64  
 5   YearBuilt      1022 non-null   int64  
 6   YearRemodAdd   1022 non-null   int64  
 7   MasVnrArea     1022 non-null   float64
 8   BsmtFinSF1     1022 non-null   int64  
 9   BsmtFinSF2     1022 non-null   int64  
 10  BsmtUnfSF      1022 non-null   int64  
 11  TotalBsmtSF    1022 non-null   int64  
 12  1stFlrSF       1022 non-null   int64  
 13  2ndFlrSF       1022 non-null   int64  
 14  LowQualFinSF   1022 non-null   int64  
 15  GrLivArea      1022 non-null   int64  
 16  BsmtFullBath   1022 non-null   int64  
 17  BsmtHalfBath   1022 non-null   int64  
 18  FullBath

In [1015]:
#do the same to test data

In [1016]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438 entries, 529 to 898
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     438 non-null    int64  
 1   MSZoning       438 non-null    object 
 2   LotFrontage    368 non-null    float64
 3   LotArea        438 non-null    int64  
 4   Street         438 non-null    object 
 5   LotShape       438 non-null    object 
 6   LandContour    438 non-null    object 
 7   Utilities      438 non-null    object 
 8   LotConfig      438 non-null    object 
 9   LandSlope      438 non-null    object 
 10  Neighborhood   438 non-null    object 
 11  Condition1     438 non-null    object 
 12  Condition2     438 non-null    object 
 13  BldgType       438 non-null    object 
 14  HouseStyle     438 non-null    object 
 15  OverallQual    438 non-null    int64  
 16  OverallCond    438 non-null    int64  
 17  YearBuilt      438 non-null    int64  
 18  YearRemo

In [1017]:
X_ob = X_test.select_dtypes(include=['object'])
X_ob.fillna('Missing', inplace=True)

C:\Users\dhruv\Anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [1018]:
X_num = X_test.select_dtypes(exclude=['object'])
X_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438 entries, 529 to 898
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     438 non-null    int64  
 1   LotFrontage    368 non-null    float64
 2   LotArea        438 non-null    int64  
 3   OverallQual    438 non-null    int64  
 4   OverallCond    438 non-null    int64  
 5   YearBuilt      438 non-null    int64  
 6   YearRemodAdd   438 non-null    int64  
 7   MasVnrArea     435 non-null    float64
 8   BsmtFinSF1     438 non-null    int64  
 9   BsmtFinSF2     438 non-null    int64  
 10  BsmtUnfSF      438 non-null    int64  
 11  TotalBsmtSF    438 non-null    int64  
 12  1stFlrSF       438 non-null    int64  
 13  2ndFlrSF       438 non-null    int64  
 14  LowQualFinSF   438 non-null    int64  
 15  GrLivArea      438 non-null    int64  
 16  BsmtFullBath   438 non-null    int64  
 17  BsmtHalfBath   438 non-null    int64  
 18  FullBath

In [1019]:
X_num = X_num.fillna(X_num.mean())
X_num.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
529,20,70.913043,32668,6,3,1957,1975,103.993103,1219,0,...,484,0,0,200,0,0,0,0,3,2007
491,50,79.000000,9490,6,7,1941,1950,0.000000,403,165,...,240,0,0,32,0,0,0,0,8,2006
459,50,70.913043,7015,5,4,1950,1950,161.000000,185,0,...,352,0,0,248,0,0,0,0,7,2009
279,60,83.000000,10005,7,5,1977,1977,299.000000,392,0,...,505,288,117,0,0,0,0,0,3,2008
655,160,21.000000,1680,6,5,1971,1971,381.000000,0,0,...,264,0,0,0,0,0,0,0,3,2010


In [1020]:
X_test = pd.concat([X_num,X_ob],axis=1,sort=False)


In [1021]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438 entries, 529 to 898
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     438 non-null    int64  
 1   LotFrontage    438 non-null    float64
 2   LotArea        438 non-null    int64  
 3   OverallQual    438 non-null    int64  
 4   OverallCond    438 non-null    int64  
 5   YearBuilt      438 non-null    int64  
 6   YearRemodAdd   438 non-null    int64  
 7   MasVnrArea     438 non-null    float64
 8   BsmtFinSF1     438 non-null    int64  
 9   BsmtFinSF2     438 non-null    int64  
 10  BsmtUnfSF      438 non-null    int64  
 11  TotalBsmtSF    438 non-null    int64  
 12  1stFlrSF       438 non-null    int64  
 13  2ndFlrSF       438 non-null    int64  
 14  LowQualFinSF   438 non-null    int64  
 15  GrLivArea      438 non-null    int64  
 16  BsmtFullBath   438 non-null    int64  
 17  BsmtHalfBath   438 non-null    int64  
 18  FullBath

In [1022]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 64 to 684
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1022 non-null   int64  
 1   LotFrontage    1022 non-null   float64
 2   LotArea        1022 non-null   int64  
 3   OverallQual    1022 non-null   int64  
 4   OverallCond    1022 non-null   int64  
 5   YearBuilt      1022 non-null   int64  
 6   YearRemodAdd   1022 non-null   int64  
 7   MasVnrArea     1022 non-null   float64
 8   BsmtFinSF1     1022 non-null   int64  
 9   BsmtFinSF2     1022 non-null   int64  
 10  BsmtUnfSF      1022 non-null   int64  
 11  TotalBsmtSF    1022 non-null   int64  
 12  1stFlrSF       1022 non-null   int64  
 13  2ndFlrSF       1022 non-null   int64  
 14  LowQualFinSF   1022 non-null   int64  
 15  GrLivArea      1022 non-null   int64  
 16  BsmtFullBath   1022 non-null   int64  
 17  BsmtHalfBath   1022 non-null   int64  
 18  FullBath

In [1023]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer

In [1024]:
X_ob = X_train.select_dtypes(include=['object'])

In [1025]:
# we can turn the previous commands into 2 functions


def calculate_top_categories(df, variable, how_many=10):
    return [
        x for x in df[variable].value_counts().sort_values(
            ascending=False).head(how_many).index
    ]


def one_hot_encode(train, test, variable, top_x_labels):

    for label in top_x_labels:
        train[variable + '_' + label] = np.where(
            train[variable] == label, 1, 0)
        
        test[variable + '_' + label] = np.where(
            test[variable] == label,1, 0)

In [1026]:
for col in X_ob.columns:
    
    top_categories = calculate_top_categories(X_train,col, how_many=10)
    
    one_hot_encode(X_train, X_test,col, top_categories)

In [1027]:
X_train.shape

(1022, 287)

In [1028]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 64 to 684
Columns: 287 entries, MSSubClass to SaleCondition_AdjLand
dtypes: float64(3), int32(213), int64(33), object(38)
memory usage: 1.4+ MB


In [1029]:
X_test.shape

(438, 287)

In [1030]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438 entries, 529 to 898
Columns: 287 entries, MSSubClass to SaleCondition_AdjLand
dtypes: float64(3), int32(213), int64(33), object(38)
memory usage: 621.1+ KB


In [1031]:
X_train = X_train.select_dtypes(exclude=['object'])
X_test  = X_test.select_dtypes(exclude=['object'])


In [1032]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 64 to 684
Columns: 249 entries, MSSubClass to SaleCondition_AdjLand
dtypes: float64(3), int32(213), int64(33)
memory usage: 1.1 MB


# FEATURE SELECTION

In [1033]:
# keep copy of original
X_train_original = X_train.copy()
X_test_original = X_test.copy()

In [1034]:
X_train_original.shape

(1022, 249)

In [1035]:
# remove constant features
constant_features = [
    feat for feat in X_train.columns if X_train[feat].std() == 0
]

X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((1022, 249), (438, 249))

In [1036]:
# remove quasi constant
sel = VarianceThreshold(
    threshold=0.01)  
sel.fit(X_train)  

sum(sel.get_support()) 

188

In [1037]:
features_to_keep = X_train.columns[sel.get_support()]

In [1038]:
# we can then remove the features like this
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train.shape, X_test.shape

((1022, 188), (438, 188))

In [1039]:
#transform the arrays back to dataframes
X_train= pd.DataFrame(X_train)
X_train.columns = features_to_keep
X_test= pd.DataFrame(X_test)
X_test.columns = features_to_keep

In [1040]:
# check for duplicated features in the training set and there are none
duplicated_feat = []
for i in range(0, len(X_train.columns)):
    if i % 10 == 0:  # this helps me understand how the loop is going
        print(i)

    col_1 = X_train.columns[i]

    for col_2 in X_train.columns[i + 1:]:
        if X_train[col_1].equals(X_train[col_2]):
            duplicated_feat.append(col_2)
            
len(duplicated_feat)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180


12

In [1041]:
# remove duplicated features
X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
X_test.drop(labels=duplicated_feat, axis=1, inplace=True)
X_train.shape, X_test.shape

((1022, 182), (438, 182))

In [1042]:
# keep a copy of the dataset up till now

X_train_basic_filter = X_train.copy()
X_test_basic_filter = X_test.copy()

In [1043]:
X_train_basic_filter.shape

(1022, 182)

In [1044]:
# find and remove correlated features which thankfully happen to only be one
def correlation(dataset, threshold):
    col_corr = set()  
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: 
                colname = corr_matrix.columns[i]  
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train, 0.8)
print('correlated features: ', len(set(corr_features)) )

correlated features:  24


In [1045]:
# this makes sense because with male and female it's either one or the other
corr_features

{'BldgType_1Fam',
 'BsmtFinType2_Missing',
 'BsmtQual_Missing',
 'CentralAir_N',
 'Electrical_FuseA',
 'ExterCond_Gd',
 'ExterQual_Gd',
 'Exterior2nd_AsbShng',
 'Exterior2nd_CmentBd',
 'Exterior2nd_HdBoard',
 'Exterior2nd_MetalSd',
 'Exterior2nd_VinylSd',
 'Exterior2nd_Wd Sdng',
 'GarageArea',
 'HouseStyle_2Story',
 'KitchenQual_Gd',
 'LandSlope_Mod',
 'LotShape_IR1',
 'MSZoning_RM',
 'Neighborhood_Somerst',
 'PavedDrive_N',
 'RoofStyle_Hip',
 'SaleCondition_Partial',
 'TotRmsAbvGrd'}

In [1046]:
# removed correlated  features
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)
X_train.shape, X_test.shape

((1022, 158), (438, 158))

In [1047]:
# keep a copy of the dataset at  this stage
X_train_corr = X_train.copy()
X_test_corr = X_test.copy()

In [1048]:
X_train_corr.shape

(1022, 158)

In [1049]:
# Remove Features with Lasso
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [1050]:
# fit a lasso and selet features, make sure to select l2
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l2'))
sel_.fit(scaler.transform(X_train), y_train)
X_train_lasso = pd.DataFrame(sel_.transform(X_train))
X_test_lasso = pd.DataFrame(sel_.transform(X_test))
X_train_lasso.columns = X_train.columns[(sel_.get_support())]
X_test_lasso.columns = X_train.columns[(sel_.get_support())]

C:\Users\dhruv\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [1051]:
X_train_lasso.shape, X_test_lasso.shape

((1022, 77), (438, 77))

In [1052]:
# This is not unreasonable actually
X_train_lasso.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 77 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MSSubClass            1022 non-null   float64
 1   LotFrontage           1022 non-null   float64
 2   OverallQual           1022 non-null   float64
 3   OverallCond           1022 non-null   float64
 4   YearRemodAdd          1022 non-null   float64
 5   MasVnrArea            1022 non-null   float64
 6   BsmtFinSF1            1022 non-null   float64
 7   BsmtUnfSF             1022 non-null   float64
 8   TotalBsmtSF           1022 non-null   float64
 9   1stFlrSF              1022 non-null   float64
 10  2ndFlrSF              1022 non-null   float64
 11  GrLivArea             1022 non-null   float64
 12  BsmtFullBath          1022 non-null   float64
 13  FullBath              1022 non-null   float64
 14  HalfBath              1022 non-null   float64
 15  BedroomAbvGr         

In [1053]:
y_train = list(y_train)
y_test =  list(y_test)

In [1054]:
print(len(y_train))
print(len(y_test))

1022
438


In [1055]:
X_train_basic_filter.shape

(1022, 182)

In [1056]:
X_train_original.shape

(1022, 249)

# MODELING & ACCURACY RESULTS

In [1057]:
#BASIC FILTER MODEL
X_train_basic_filter = sm.add_constant(X_train_basic_filter)
model = sm.OLS(y_train,X_train_basic_filter).fit()
#predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.907
Model:                            OLS   Adj. R-squared:                  0.889
Method:                 Least Squares   F-statistic:                     48.86
Date:                Sun, 17 May 2020   Prob (F-statistic):               0.00
Time:                        15:33:15   Log-Likelihood:                -11750.
No. Observations:                1022   AIC:                         2.384e+04
Df Residuals:                     851   BIC:                         2.468e+04
Df Model:                         170                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -4.325e+

In [1058]:
#BASIC FILTER MODEL
X_test_basic_filter = sm.add_constant(X_test_basic_filter) # adding a constant
model = sm.OLS(y_test,X_test_basic_filter).fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.878
Method:                 Least Squares   F-statistic:                     19.84
Date:                Sun, 17 May 2020   Prob (F-statistic):           1.14e-93
Time:                        15:33:15   Log-Likelihood:                -5013.1
No. Observations:                 438   AIC:                         1.036e+04
Df Residuals:                     270   BIC:                         1.105e+04
Df Model:                         167                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -2310.64

In [1062]:
#ORIGINAL MODEL
X_train_original = sm.add_constant(X_train_original)
model = sm.OLS(y_train,X_train_original).fit()
#predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     62.15
Date:                Sun, 17 May 2020   Prob (F-statistic):               0.00
Time:                        15:33:16   Log-Likelihood:                -11525.
No. Observations:                1022   AIC:                         2.346e+04
Df Residuals:                     815   BIC:                         2.448e+04
Df Model:                         206                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -4.746e+

In [1063]:
# ORIGINAL MODEL
X_test_original = sm.add_constant(X_test_original)
model = sm.OLS(y_train,X_train_original).fit()
#predictions = model.predict(X) 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     62.15
Date:                Sun, 17 May 2020   Prob (F-statistic):               0.00
Time:                        15:33:16   Log-Likelihood:                -11525.
No. Observations:                1022   AIC:                         2.346e+04
Df Residuals:                     815   BIC:                         2.448e+04
Df Model:                         206                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -4.746e+

In [1064]:
basicresults =  pd.concat([y_test,pred_df], axis=1, sort=False)

In [1065]:
#CNP_xtrain = sm.add_constant(CNP_xtrain, has_constant='add') # adding a constant
X_train_corr = sm.add_constant(X_train_corr)
model = sm.OLS(y_train,X_train_corr).fit()
#predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     52.81
Date:                Sun, 17 May 2020   Prob (F-statistic):               0.00
Time:                        15:33:16   Log-Likelihood:                -11775.
No. Observations:                1022   AIC:                         2.386e+04
Df Residuals:                     869   BIC:                         2.461e+04
Df Model:                         152                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  3.388e+

In [1066]:
#CNP_xtrain = sm.add_constant(CNP_xtrain, has_constant='add') # adding a constant
X_test_corr = sm.add_constant(X_test_corr)
model = sm.OLS(y_test,X_test_corr).fit()
#predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                 Prices   R-squared:                       0.913
Model:                            OLS   Adj. R-squared:                  0.868
Method:                 Least Squares   F-statistic:                     19.98
Date:                Sun, 17 May 2020   Prob (F-statistic):           4.16e-96
Time:                        15:33:17   Log-Likelihood:                -5043.6
No. Observations:                 438   AIC:                         1.039e+04
Df Residuals:                     286   BIC:                         1.101e+04
Df Model:                         151                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                 -6.773e+

In [1067]:
#CNP_xtrain = sm.add_constant(CNP_xtrain, has_constant='add') # adding a constant
X_train_lasso = sm.add_constant(X_train_lasso)
model = sm.OLS(y_train,X_train_lasso).fit()
#predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.875
Method:                 Least Squares   F-statistic:                     93.90
Date:                Sun, 17 May 2020   Prob (F-statistic):               0.00
Time:                        15:33:17   Log-Likelihood:                -11861.
No. Observations:                1022   AIC:                         2.388e+04
Df Residuals:                     944   BIC:                         2.426e+04
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 7.721e+05 

In [1068]:
#CNP_xtrain = sm.add_constant(CNP_xtrain, has_constant='add') # adding a constant
X_test_lasso = sm.add_constant(X_test_lasso)
model = sm.OLS(y_test,X_test_lasso).fit()
#predictions = model.predict(X) 
 
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                 Prices   R-squared:                       0.865
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     29.89
Date:                Sun, 17 May 2020   Prob (F-statistic):          3.49e-117
Time:                        15:33:17   Log-Likelihood:                -5141.2
No. Observations:                 438   AIC:                         1.044e+04
Df Residuals:                     360   BIC:                         1.076e+04
Df Model:                          77                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.223e+06 

NOW WE CAN TEST MODEL ACCURACY

In [1069]:
pred = model.get_prediction(X_test_lasso)
pred_df = pred.summary_frame()

In [1070]:
pred_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,233321.965739,19534.451648,194905.992498,271737.938981,157183.340139,309460.591340
1,153516.953992,12566.349801,128804.278849,178229.629135,83288.614834,223745.293151
2,92400.220687,18685.249297,55654.267950,129146.173423,17090.404920,167710.036453
3,197960.044462,13295.207727,171814.015208,224106.073716,127214.598290,268705.490633
4,91533.981942,12505.565154,66940.844393,116127.119491,21347.617719,161720.346164
...,...,...,...,...,...,...
433,245039.030699,16536.914558,212517.940342,277560.121057,171697.878361,318380.183038
434,184593.304946,15071.124976,154954.800411,214231.809482,112484.052701,256702.557192
435,321117.156989,13050.714925,295451.941279,346782.372699,250547.995129,391686.318850
436,194889.451620,9192.332679,176812.035750,212966.867490,126712.503369,263066.399871


In [1071]:
from pandas import DataFrame
y_test = DataFrame(y_test,columns=['Prices'])

In [1072]:
y_test

,Prices
0,200624
1,133000
2,110000
3,192000
4,88000
...,...
433,241500
434,127500
435,350000
436,227000


In [1073]:
result = pd.concat([y_test,pred_df], axis=1, sort=False)

In [1074]:
result.head()

,Prices,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,200624,233321.965739,19534.451648,194905.992498,271737.938981,157183.340139,309460.591340
1,133000,153516.953992,12566.349801,128804.278849,178229.629135,83288.614834,223745.293151
2,110000,92400.220687,18685.249297,55654.267950,129146.173423,17090.404920,167710.036453
3,192000,197960.044462,13295.207727,171814.015208,224106.073716,127214.598290,268705.490633
4,88000,91533.981942,12505.565154,66940.844393,116127.119491,21347.617719,161720.346164


In [1075]:
result.to_excel('lassoregressiontest.xlsx')


In [1076]:
from statistics import mean

In [1077]:
result['PE'] = abs(result['Prices'] - result['mean'])/result['Prices']
print(mean(result['PE'] )) 

0.11380356668622432


In [1078]:
result

,Prices,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,PE
0,200624,233321.965739,19534.451648,194905.992498,271737.938981,157183.340139,309460.591340,0.162981
1,133000,153516.953992,12566.349801,128804.278849,178229.629135,83288.614834,223745.293151,0.154263
2,110000,92400.220687,18685.249297,55654.267950,129146.173423,17090.404920,167710.036453,0.159998
3,192000,197960.044462,13295.207727,171814.015208,224106.073716,127214.598290,268705.490633,0.031042
4,88000,91533.981942,12505.565154,66940.844393,116127.119491,21347.617719,161720.346164,0.040159
...,...,...,...,...,...,...,...,...
433,241500,245039.030699,16536.914558,212517.940342,277560.121057,171697.878361,318380.183038,0.014654
434,127500,184593.304946,15071.124976,154954.800411,214231.809482,112484.052701,256702.557192,0.447791
435,350000,321117.156989,13050.714925,295451.941279,346782.372699,250547.995129,391686.318850,0.082522
436,227000,194889.451620,9192.332679,176812.035750,212966.867490,126712.503369,263066.399871,0.141456


In [1079]:
def mape(result): 
    result['PE'] = abs(result['Prices'] - result['mean'])/result['Prices']
    print(mean(result['PE'] )) 
    

In [1080]:
mape(result)

0.11380356668622432
